In [74]:
from pyspark.sql.types import *
from pyspark.sql.functions import expr, col, desc, lower, instr

In [75]:
google_play_df = spark.read.csv('../data/googleplaystore.csv', header=True, inferSchema=True, nullValue='', sep=',')

In [76]:
schema = StructType( [
    StructField('App', StringType()),
    StructField('Translated Review', StringType()),
    StructField('Sentiment', StringType() ),
    StructField('Sentiment_Polarity', DoubleType()),
    StructField('Sentiment_Subjectivity', DoubleType())
] )
app_reviews_df = spark.read.csv('../data/googleplaystore_user_reviews.csv', sep=',', schema=schema, header=True)

In [77]:
google_play_df = google_play_df.withColumn('Price', expr('double(replace(Price,"\$",""))'))
google_play_df = google_play_df.withColumn('Installs', expr('double(replace(replace(Installs, "+", ""), ",", ""))') )
google_play_df=google_play_df.withColumn('Size', expr('double(replace(replace(Size,"M","000"),"K",""))'))
google_play_df=google_play_df.drop('Android Ver')
google_play_df=google_play_df.drop('Genres')
google_play_df=google_play_df.drop('Last Updated')
google_play_df=google_play_df.drop('Current Ver')

1. Ejercicio 1. Usamos un filter para filtrar por la condición dada y nos quedamos con las columnas App y Category

In [78]:
google_play_df_1 = google_play_df.filter(expr('Rating>=4.2')).select('App', 'Category')
google_play_df_1.show(5)

+--------------------+--------------+
|                 App|      Category|
+--------------------+--------------+
|U Launcher Lite –...|ART_AND_DESIGN|
|Sketch - Draw & P...|ART_AND_DESIGN|
|Pixel Draw - Numb...|ART_AND_DESIGN|
|Paper flowers ins...|ART_AND_DESIGN|
|Garden Coloring Book|ART_AND_DESIGN|
+--------------------+--------------+
only showing top 5 rows



2. Ejercicio 2. Usamos filter para filtrar por la cndición y con count obtenemos el número de registros

In [79]:
google_play_df_2_count = google_play_df.filter(expr('Size>10000')).count()
google_play_df_2_count

5085

3. Ejercicio 3. Con dropna eliminamos las filas que tengan un valor nulo en Type o Category. Agrupamos por Type, Category y Content Rating y por cada tripleta calculamos la media de Rating

In [80]:
google_play_df_3 = google_play_df.dropna('any', subset=['Type', 'Category']).groupby(col('Type'), col('Category'), col('Content Rating')).agg(expr('mean(Rating)'))
google_play_df_3.show()

+----+-------------------+--------------+------------------+
|Type|           Category|Content Rating|      mean(Rating)|
+----+-------------------+--------------+------------------+
|Free|             DATING|          Teen|3.5999999999999996|
|Free|          EDUCATION|      Everyone| 4.374820143884892|
|Free|       PRODUCTIVITY|  Everyone 10+|               3.6|
|Free|      ENTERTAINMENT|  Everyone 10+|               4.1|
|Free|          LIFESTYLE|      Everyone| 4.070037453183519|
|Paid|          EDUCATION|      Everyone|              4.75|
|Free|              TOOLS|    Mature 17+|               3.7|
|Paid|          LIFESTYLE|          Teen|               5.0|
|Free|BOOKS_AND_REFERENCE|          Teen|4.3050000000000015|
|Free|    PERSONALIZATION|      Everyone|4.2849514563106785|
|Free|             DATING|      Everyone| 4.266666666666667|
|Free|     FOOD_AND_DRINK|      Everyone| 4.142857142857143|
|Free|        PHOTOGRAPHY|      Everyone| 4.193971631205671|
|Free|   TRAVEL_AND_LOCA

4. Ejercicio 4. Con dropna quitamos las filas con valores nulos en el campo Type. Usamos crosstab para crear la tabla de contigencia entre Content Rating y Type

In [81]:
google_play_df_4 = google_play_df.dropna('any', subset=['Type']).crosstab('Content Rating', 'Type')
google_play_df_4.show()

+-------------------+----+----+
|Content Rating_Type|Free|Paid|
+-------------------+----+----+
|            Unrated|   2|   0|
|         Mature 17+| 479|  20|
|       Everyone 10+| 380|  33|
|    Adults only 18+|   3|   0|
|           Everyone|8020| 695|
|               Teen|1156|  52|
+-------------------+----+----+



5. Ejercicio 5. Creamos un dataframe temporal que agrupe por categoría y calcule la media de Rating por cada categoría. Unimos este dataframe con el original por el campo Category y con flter filtranos aquellos registros cuyo Rating es mayor que la media de Ratings de su categoría. Tras esto, nos quedamos con el campo App a través de select

In [82]:
google_play_df_5_temp = google_play_df.groupby(col('Category')).agg(expr('mean(Rating)')).withColumnRenamed('Category', 'CategoryR')
google_play_df_5 = google_play_df.join(google_play_df_5_temp, google_play_df.Category == google_play_df_5_temp.CategoryR, how='inner').drop('CategoryR').filter(col('Rating') > col('mean(Rating)')).select('App')
google_play_df_5.show()

+--------------------+
|                 App|
+--------------------+
|U Launcher Lite –...|
|Sketch - Draw & P...|
|Paper flowers ins...|
|Garden Coloring Book|
|Kids Paint Free -...|
|Text on Photo - F...|
|Name Art Photo Ed...|
|Mandala Coloring ...|
|3D Color Pixel by...|
|Photo Designer - ...|
|350 Diy Room Deco...|
|        ibis Paint X|
|Superheroes Wallp...|
|HD Mickey Minnie ...|
|Harley Quinn wall...|
|Colorfit - Drawin...|
|Anime Manga Color...|
|     I Creative Idea|
|UNICORN - Color B...|
|PIP Camera - PIP ...|
+--------------------+
only showing top 20 rows



6. Ejercicio 6. Primero, obtenemos aquellas aplicaciones que tengan más de 300 reviews a través de filter. Por otro lado, obtenemos las reviews cuya polaridad sea 1. Unimos estos dos datasets de modo que tengamos tantas filas como reviews filtradas, y mediante dropDuplicates, quitamos aquellas filas en las que se repita App (ya que hay aplicaciones con más de un comentario con polaridad = 1). Tras esto, nos quedamos con los campos App y Category

In [83]:
google_play_df_6_temp = google_play_df.filter(expr('Reviews>300')).withColumnRenamed('App', 'AppR')
app_reviews_df_6 = app_reviews_df.filter(expr('Sentiment_Polarity=1'))
google_play_df_6 = app_reviews_df_6.join(google_play_df_6_temp, app_reviews_df_6.App == google_play_df_6_temp.AppR, how="inner").dropDuplicates(subset=['App']).select('App', 'Category')
google_play_df_6.show()

+--------------------+------------------+
|                 App|          Category|
+--------------------+------------------+
|  Floor Plan Creator|    ART_AND_DESIGN|
|AOL - News, Mail ...|NEWS_AND_MAGAZINES|
|      Epocrates Plus|           MEDICAL|
|       File Explorer|      PRODUCTIVITY|
|            Arrow.io|              GAME|
|CALCU™ Stylish Ca...|             TOOLS|
|FilterGrid - Cam&...|       PHOTOGRAPHY|
|BELONG Beating Ca...|           MEDICAL|
|      Bubble Shooter|              GAME|
|Football Live Scores|            SPORTS|
|     Amazon Shopping|          SHOPPING|
|Basketball FRVR -...|            FAMILY|
|CM Launcher 3D - ...|   PERSONALIZATION|
|Dog Licks Screen ...|   PERSONALIZATION|
|Drugs.com Medicat...|           MEDICAL|
|           A+ Mobile|           FINANCE|
|             Dropbox|      PRODUCTIVITY|
|2GIS: directory &...|  TRAVEL_AND_LOCAL|
|         Alarm Clock|             TOOLS|
|       Baseball Boy!|              GAME|
+--------------------+------------

7. Ejercicio 7. Con dropna, quitamos aquellas filas en las que en el campo Translated Review sea nulo. Tras esto, mapeamos con withColumn dicho campo a su mismo valor usando minúsculas con la función lower. Creamos una columna hasGreat que, mediante la función instr, determine si contiene la ristra 'great'. Filtramos aquellasfilas cuyo valor de hasgReat sea mayor que 0, ya que un 0 indica que no encontró coincidencias con 'great'. Tras esto, con dropDuplicates nos quedamos con una fila por aplicación, y nos quedamos con el campo App.

In [84]:
app_reviews_df_7 = app_reviews_df.dropna('any', subset=['Translated Review']).withColumn('Translated Review', lower(col('Translated Review'))).withColumn('hasGreat', instr(col('Translated Review'), 'great')).filter(expr('hasGreat>0')).dropDuplicates(subset=['App']).select('App')
app_reviews_df_7.show()

+--------------------+
|                 App|
+--------------------+
|BaBe+ - Berita In...|
|    Basketball Stars|
|Davis's Drug Guid...|
|Find&Save - Local...|
|  Floor Plan Creator|
|     Candy Pop Story|
|Cricbuzz - Live C...|
|        Google Earth|
|    Homework Planner|
|      Epocrates Plus|
|            Arrow.io|
|CALCU™ Stylish Ca...|
|FilterGrid - Cam&...|
|Golf GPS Rangefin...|
|BELONG Beating Ca...|
|      Bubble Shooter|
|Football Live Scores|
|     Amazon Shopping|
|CM Launcher 3D - ...|
|Dream League Socc...|
+--------------------+
only showing top 20 rows



8. Ejercicio 8. Con la función corr obtenemos el coeficiente de correlación de Pearson

In [85]:
corr_8 = google_play_df.corr('Rating', 'Size')
corr_8

0.10658246952806998

La correlación nos da un valor de 0.1066, lo que significa que las columnas están muy poco correlacionadas, ya que no se acerca ni a 1 ni a -1.

9. Ejercicio 9. Unimos el dataset de las aplicaciones con el de las reviews a través del capo App- Nos quedamos con el campo Content Rating y Sentiment_Polarity. Agrupamos por Content Rating y sobre esta agrupación, calculamos la media y la desviación estándar de Sentiment_Polarity con mean y stddev_samp respectivamente

In [86]:
google_play_df_9 = google_play_df.select('App', 'Content Rating').withColumnRenamed('App', 'AppR')
joined_df_9 = app_reviews_df.join(google_play_df_9, app_reviews_df.App == google_play_df_9.AppR, how="inner").select('Content Rating', 'Sentiment_Polarity').groupby(col('Content Rating')).agg(expr('mean(Sentiment_Polarity)'), expr('stddev_samp(Sentiment_Polarity)'))
joined_df_9.show()

+---------------+------------------------+-------------------------------+
| Content Rating|mean(Sentiment_Polarity)|stddev_samp(Sentiment_Polarity)|
+---------------+------------------------+-------------------------------+
|           Teen|     0.12102082880135945|            0.30982467421170184|
|     Mature 17+|     0.16858084745627555|             0.3678941965284479|
|   Everyone 10+|     0.13200213901805052|            0.31099070220385955|
|       Everyone|     0.16477698031357257|            0.33459519837814994|
|Adults only 18+|     0.31233355379188715|              0.334873921904287|
+---------------+------------------------+-------------------------------+



10. Ejercicio 10. Creamos la tabla temporal con createOrReplaceTempView

In [87]:
google_play_df.withColumnRenamed('Content Rating', 'Content_Rating').createOrReplaceTempView('GooglePlay')

11. Ejercicio 11. Creamos la tabla temporal con createOrReplaceTempView

In [42]:
app_reviews_df.createOrReplaceTempView('Reviews')

12. Ejercicio 12

In [50]:
google_play_df_12 = spark.sql('SELECT App, Category FROM GooglePlay WHERE Rating >= 4.2')
google_play_df_12.show(5)

+--------------------+--------------+
|                 App|      Category|
+--------------------+--------------+
|U Launcher Lite –...|ART_AND_DESIGN|
|Sketch - Draw & P...|ART_AND_DESIGN|
|Pixel Draw - Numb...|ART_AND_DESIGN|
|Paper flowers ins...|ART_AND_DESIGN|
|Garden Coloring Book|ART_AND_DESIGN|
+--------------------+--------------+
only showing top 5 rows



13. Ejercicio 13. Cabe destacar el uso de AVG(Rating) que calcula la media de este campo y GROUP BY, que permite agrupar por campos

In [88]:
google_play_df_13 = spark.sql('SELECT Type, Category, Content_Rating, AVG(Rating) FROM GooglePlay WHERE Type is not NULL AND Category is not NULL GROUP BY Type, Category, Content_Rating')
google_play_df_13.show()

+----+-------------------+--------------+------------------+
|Type|           Category|Content_Rating|       avg(Rating)|
+----+-------------------+--------------+------------------+
|Free|             DATING|          Teen|3.5999999999999996|
|Free|          EDUCATION|      Everyone| 4.374820143884892|
|Free|       PRODUCTIVITY|  Everyone 10+|               3.6|
|Free|      ENTERTAINMENT|  Everyone 10+|               4.1|
|Free|          LIFESTYLE|      Everyone| 4.070037453183519|
|Paid|          EDUCATION|      Everyone|              4.75|
|Free|              TOOLS|    Mature 17+|               3.7|
|Paid|          LIFESTYLE|          Teen|               5.0|
|Free|BOOKS_AND_REFERENCE|          Teen|4.3050000000000015|
|Free|    PERSONALIZATION|      Everyone|4.2849514563106785|
|Free|             DATING|      Everyone| 4.266666666666667|
|Free|     FOOD_AND_DRINK|      Everyone| 4.142857142857143|
|Free|        PHOTOGRAPHY|      Everyone| 4.193971631205671|
|Free|   TRAVEL_AND_LOCA

14. Ejercicio 14. Cabe destacar el uso de STD(Sentiment_Polarity), que permite calcular la desviación típica de esta columna

In [89]:
google_play_df_14 = spark.sql('SELECT Content_Rating, AVG(Sentiment_Polarity), STD(Sentiment_Polarity) FROM Reviews INNER JOIN GooglePlay ON Reviews.App=GooglePlay.App GROUP BY Content_Rating')
google_play_df_14.show()

+---------------+-----------------------+-------------------------------+
| Content_Rating|avg(Sentiment_Polarity)|stddev_samp(Sentiment_Polarity)|
+---------------+-----------------------+-------------------------------+
|           Teen|    0.12102082880135945|            0.30982467421170184|
|     Mature 17+|    0.16858084745627555|             0.3678941965284479|
|   Everyone 10+|    0.13200213901805052|            0.31099070220385955|
|       Everyone|    0.16477698031357257|            0.33459519837814994|
|Adults only 18+|    0.31233355379188715|              0.334873921904287|
+---------------+-----------------------+-------------------------------+



Eliminamos las tablas

In [90]:
spark.catalog.dropTempView('GooglePlay')
spark.catalog.dropTempView('Reviews')